<a href="https://colab.research.google.com/github/sefeoglu/AE_Parseval_Network/blob/master/src/notebooks/FGSM_WResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

In [0]:
from sklearn.model_selection import train_test_split
import gzip
import pickle
import numpy as np

In [239]:
def read_data():
    with open("data.pz", 'rb') as file_:
        with gzip.GzipFile(fileobj=file_) as gzf:
            data = pickle.load(gzf, encoding='latin1', fix_imports=True)
    return data
data = read_data()
new_data_X = []
Y_data = []
for row in data:
    new_data_X.append(row['crop'])
    Y_data.append(row['label'])
new_data_X = np.array(new_data_X)
new_data_X.shape

(5722, 68, 100)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(new_data_X, Y_data, test_size=0.33, random_state=42)

In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
# creating initial dataframe

y_train_df = pd.DataFrame(y_train, columns=['Label'])
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
y_train_df['New'] = labelencoder.fit_transform(y_train_df['Label'])
y_test_df = pd.DataFrame(y_test, columns=['Label'])
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
y_test_df['New'] = labelencoder.fit_transform(y_test_df['Label'])

In [0]:
import keras.backend as K
img_rows, img_cols = X_train[0].shape


# transform data set
if K.common.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [0]:
from keras.utils import to_categorical
y_test = to_categorical(y_test_df['New'])
y_train = to_categorical(y_train_df['New'])

In [0]:
import keras

model = keras.models.load_model("wresnet.h5")

Adversarial Attack


In [246]:
model.input.shape

TensorShape([None, 68, 100, 1])

In [0]:
def preprocess(image):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, (68, 100))
  image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
  image = image[None, ...]
  return image

# Helper function to extract labels from probability vector

In [0]:
img = X_train[8]

In [0]:
import tensorflow as tf
image = tf.convert_to_tensor(img)
image = preprocess(image)

In [0]:
def model_loss(y, model, mean=True):
    """
    Define loss of TF graph
    :param y: correct labels
    :param model: output of the model
    :param mean: boolean indicating whether should return mean of loss
                 or vector of losses for each input of the batch
    :return: return mean of loss if True, otherwise return vector with per
             sample loss
    """

    op = model.op

    if "softmax" in str(op).lower():
        logits, = op.inputs
    else:
        logits = model
    out = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)

    if mean:
        out = tf.reduce_mean(out)

    return out


(<tf.Tensor 'model_1_9/dense_1/BiasAdd:0' shape=(1, 4) dtype=float32>,)

In [276]:

loss = model_loss(label, model(image), mean=False)
grad, = keras.backend.gradients(loss, image)
print(grad)

Tensor("model_1_9/dense_1/BiasAdd:0", shape=(1, 4), dtype=float32)
Tensor("Reshape_25:0", shape=(1,), dtype=float32)
Tensor("gradients_30/model_1_9/conv2d_1/convolution_grad/Conv2DBackpropInput:0", shape=(1, 68, 100, 1), dtype=float32)


In [324]:
from six.moves import xrange
red_ind = list(xrange(1, len(image.get_shape())))
square = tf.reduce_sum(tf.square(grad))
normalized_grad = grad / tf.sqrt(square)
normalized_grad

<tf.Tensor 'RealDiv_10:0' shape=(1, 68, 100, 1) dtype=float32>

In [336]:
from nets.resnet_50 import resnet50
cl

52

In [306]:
scaled_grad = 0.3 * normalized_grad
adv = scaled_grad+image
tf.compat

<tf.Operation 'AddV2_13' type=AddV2>

In [250]:
probs = model.predict(image,batch_size=None,steps=1)
labrador_retriever_index = 208
label = tf.one_hot(3, probs.shape[-1])
label = tf.reshape(label, (1, probs.shape[-1]))
label

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0., 0., 0., 1.]], dtype=float32)>

In [0]:
probs = tf.convert_to_tensor(probs)

In [0]:
probs

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.02893856, 0.22029065, 0.09846024, 0.65231055]], dtype=float32)>

In [0]:
 l = keras.losses.categorical_crossentropy(label, probs)

In [0]:
print(l)

tf.Tensor([0.42723453], shape=(1,), dtype=float32)


In [0]:
loss_object = keras.losses.CategoricalCrossentropy()
variable = tf.Variable(image)
np.gradient(np.array([l]))

ValueError: ignored

In [0]:
generate_pattern("dense_")


NameError: ignored